###  TODO  Exercise 7. ex
RENOIR
### You are given a ”renoir” set of two images (reference and noisy) from the RENOIR dataset 2 .
(a) Visualise all color channels of both images. 
Are the all channels equally affected by the noise?
<br>

In [ ]:
import scipy.ndimage as ndimage
import matplotlib.pyplot as plt
import cv2
import numpy as np

im1 = ndimage.imread("./data/renoir/Noisy.bmp")
im2 = ndimage.imread("./data/renoir/Reference.bmp", -1)

plt.imshow(im1)
plt.show()
plt.imshow(im2)
plt.show()

red_images = im1[:,:,0]
green_images = im1[:,:,1]
blue_images = im1[:,:,2]

img = cv2.imread('home.jpg',0)
plt.hist(red_images,256,[0,256]); 
plt.show()

plt.plot(
    np.ndarray.flatten(green_images)
)
plt.show()
plt.plot(
    np.ndarray.flatten(blue_images)
)
plt.show()
    
plt.ylabel('pixel ')
plt.xlabel('value ')
plt.show()
plt.plot(
    np.ndarray.flatten(im2)
)
plt.show()


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.
  import sys


<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

(b) Try to decrease the noise via image down/up sampling.
<br>

(a) Do it for the RGB image. Measure the PSNR between the reference and de-noised
images 3 .
<br>

(b) Do it for the grayscale image. Measure the PSNR between the grayscale reference
and de-noised images.
<br>

(c) Do the denoising for the RGB image and after convert it to the grayscale. Measure
the PSNR. Does the obtained result is different from the (b)? Explain the r
<br>